In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling PProf [e4faabce-9ead-11e9-39d9-4379958e3056]
└ @ Base loading.jl:1260


In [2]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate() + LoggerMessageGate())
    
    x_prev_add = add!(model, AdditionNode())
    
    x_prior = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1   = add!(model, constvar(:add_1, 1.0))

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    
    add_x_and_noise = add!(model, AdditionNode())
    
    x = add!(model, simplerandomvar(:x))
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = add!(model, datavar(:y, Float64))
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(model, x_prev_add)
    activate!(model, add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getmarginal(x))
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [3]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [4]:
kalman(data)

From variable out of node + => Normal{Float64}(μ=1.0, σ=100.0)
From variable in1 of node + => Normal{Float64}(μ=0.16150074734325148, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=1.1779419091600505, σ=14.002800840280099)
From variable in1 of node + => Normal{Float64}(μ=-2.9968563545985676, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=0.11121009541816052, σ=9.950371902099892)
From variable in1 of node + => Normal{Float64}(μ=6.269297146334534, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=3.1503117679070263, σ=8.137884587711596)
From variable in1 of node + => Normal{Float64}(μ=6.604668270495032, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=5.0096044302423515, σ=7.053456158585984)
From variable in1 of node + => Normal{Float64}(μ=3.6200509046103972, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=5.732800939080608, σ=6.311944030978033)
From variable in1 of node + => 

10-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.17794190916005048, σ=14.002800840280099)
 Normal{Float64}(μ=-0.8887899045818395, σ=9.950371902099892)
 Normal{Float64}(μ=2.1503117679070263, σ=8.137884587711596)
 Normal{Float64}(μ=4.0096044302423515, σ=7.053456158585984)
 Normal{Float64}(μ=4.732800939080608, σ=6.311944030978033)
 Normal{Float64}(μ=5.087108838176777, σ=5.763904177042351)
 Normal{Float64}(μ=6.896369236570357, σ=5.337605126836238)
 Normal{Float64}(μ=8.147419660450211, σ=4.993761694389224)
 Normal{Float64}(μ=6.958525396417643, σ=4.708816093480111)
 Normal{Float64}(μ=9.234943249006411, σ=4.467670516087703)